## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,75.16,79,93,9.95,overcast clouds
1,1,Barrow,US,71.2906,-156.7887,-11.18,77,90,5.75,overcast clouds
2,2,Iqaluit,CA,63.7506,-68.5145,16.25,75,97,7.47,overcast clouds
3,3,Saskylakh,RU,71.9167,114.0833,-44.19,100,66,3.49,broken clouds
4,4,Carnarvon,AU,-24.8667,113.6333,68.07,83,40,20.71,scattered clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,75.16,79,93,9.95,overcast clouds
9,9,Hilo,US,19.7297,-155.0900,83.28,75,20,5.75,few clouds
12,12,Saint-Pierre,RE,-21.3393,55.4781,76.68,87,90,18.41,overcast clouds
15,15,Terra Santa,BR,-2.1042,-56.4869,84.36,73,55,1.48,light rain
19,19,Castro,BR,-24.7911,-50.0119,73.89,64,23,8.25,moderate rain
20,20,Biak,ID,-0.9131,122.8766,77.65,75,100,2.01,overcast clouds
23,23,Butaritari,KI,3.0707,172.7902,81.30,75,85,13.82,overcast clouds
24,24,Edea,CM,3.8000,10.1333,77.45,91,100,2.66,overcast clouds
26,26,Mendoza,AR,-32.8908,-68.8272,88.21,25,58,7.72,broken clouds
28,28,Amalapuram,IN,16.5833,82.0167,71.78,83,100,4.81,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                271
City                   271
Country                270
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Current Description    271
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pd.DataFrame(clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]).copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
9,Hilo,US,83.28,few clouds,19.7297,-155.0900,
12,Saint-Pierre,RE,76.68,overcast clouds,-21.3393,55.4781,
15,Terra Santa,BR,84.36,light rain,-2.1042,-56.4869,
19,Castro,BR,73.89,moderate rain,-24.7911,-50.0119,
20,Biak,ID,77.65,overcast clouds,-0.9131,122.8766,
23,Butaritari,KI,81.30,overcast clouds,3.0707,172.7902,
24,Edea,CM,77.45,overcast clouds,3.8000,10.1333,
26,Mendoza,AR,88.21,broken clouds,-32.8908,-68.8272,
28,Amalapuram,IN,71.78,overcast clouds,16.5833,82.0167,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [19]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Hilo,US,83.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Saint-Pierre,RE,76.68,overcast clouds,-21.3393,55.4781,Lindsey Hôtel
15,Terra Santa,BR,84.36,light rain,-2.1042,-56.4869,Hotel Sarah
19,Castro,BR,73.89,moderate rain,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
678,Haapiti,PF,83.93,light rain,-17.5667,-149.8667,Linareva Moorea Beach Resort
679,Ikorodu,NG,82.36,overcast clouds,6.6191,3.5041,Ibisun Hotel & Suites
683,Innisfail,AU,77.05,overcast clouds,-17.5333,146.0333,Moondarra Motel
684,Mercedes,AR,84.18,clear sky,-34.6515,-59.4307,Posada del Inti Centro


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.DataFrame(hotel_df).dropna()

In [33]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Hilo,US,83.28,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Saint-Pierre,RE,76.68,overcast clouds,-21.3393,55.4781,Lindsey Hôtel
15,Terra Santa,BR,84.36,light rain,-2.1042,-56.4869,Hotel Sarah
19,Castro,BR,73.89,moderate rain,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
678,Haapiti,PF,83.93,light rain,-17.5667,-149.8667,Linareva Moorea Beach Resort
679,Ikorodu,NG,82.36,overcast clouds,6.6191,3.5041,Ibisun Hotel & Suites
683,Innisfail,AU,77.05,overcast clouds,-17.5333,146.0333,Moondarra Motel
684,Mercedes,AR,84.18,clear sky,-34.6515,-59.4307,Posada del Inti Centro


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))